In [ ]:
import itertools as it
import operator
import re
import subprocess
import tempfile
import time
from collections import Counter
from functools import partial
from pathlib import Path

import duckdb
import gfapy
import holoviews as hv
import ibis
import matplotlib.pyplot as plt
import numba
import numpy as np
import pandas as pd
import parasail
import polars as pl
import pyabpoa
import pyarrow as pa
import pyarrow.compute as pc
import pyarrow.dataset as ds
import pyfastx
import pysam
import spoa
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from pyarrow import csv
from pywfa import WavefrontAligner
from tqdm.auto import tqdm, trange

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import paulssonlab.sequencing.align as align
import paulssonlab.sequencing.cigar as scigar
import paulssonlab.sequencing.consensus as con
import paulssonlab.sequencing.gfa as sgfa
import paulssonlab.sequencing.io as sio
import paulssonlab.sequencing.processing as processing
from paulssonlab.util.sequence import reverse_complement

In [ ]:
hv.extension("bokeh")

In [ ]:
%load_ext pyinstrument
import line_profiler
import pyinstrument

%load_ext line_profiler

In [ ]:
pl.enable_string_cache()

# Config

In [ ]:
gfa_filename = "/home/jqs1/scratch/jqs1/sequencing/230930_alignment_test/230707_repressilators/pLIB419.gfa"

In [ ]:
gfa = gfapy.Gfa.from_file(gfa_filename)

# Hash paths

In [ ]:
from glob import glob

In [ ]:
files = list(
    Path(f)
    for f in glob(
        "/n/scratch3/groups/hms/sysbio/paulsson/jqs1/sequencing/230818_bcd_rbses/20230818_1343_1A_PAQ97606_f49ab41c/work/c8/219d3a965f5975d7ba26c214afc5b1/input_unhashed/*.arrow"
    )
)

In [ ]:
len(files)

In [ ]:
dest_dir = files[0].parent / "input"
dest_dir.mkdir(exist_ok=True)

In [ ]:
%%time
for f in tqdm(files):
    df = pl.scan_ipc(f)
    df = df.with_columns(pl.col("path").hash().alias("path_hash"))
    df.collect().write_ipc(dest_dir / f.name)

# Prepare for FISH

In [ ]:
df = pl.read_ipc(
    "/home/jqs1/scratch/jqs1/sequencing/230707_repressilators/20230707_2040_MN35044_FAS94231_25542e0d/extract_segments/*.arrow"
)

In [ ]:
len(df)

In [ ]:
df.estimated_size(unit="mb")

In [ ]:
df.head(10)

In [ ]:
from glob import glob

In [ ]:
df.schema["consensus_seq"]

In [ ]:
df = pl.read_ipc(
    "/n/scratch3/groups/hms/sysbio/paulsson/jqs1/sequencing/230707_repressilators/20230707_2040_MN35044_FAS94231_25542e0d/work/ee/e572fbaa90b78d7765db9eb1d47a12/consensus-18-of-300.arrow"
)

In [ ]:
df

In [ ]:
?pl.Series

In [ ]:
list(
    glob(
        "/home/jqs1/scratch/jqs1/sequencing/230707_repressilators/20230707_2040_MN35044_FAS94231_25542e0d/output/default/extract_segments/*.arrow"
    )
)

In [ ]:
files = [
    pl.read_ipc(f)
    for f in glob(
        "/home/jqs1/scratch/jqs1/sequencing/230707_repressilators/20230707_2040_MN35044_FAS94231_25542e0d/output/default/extract_segments/*.arrow"
    )
]

In [ ]:
[len(f.columns) for f in files]

In [ ]:
df = files[2]

In [ ]:
files[2].get_column("consensus_path")[0]

In [ ]:
df.get_column("consensus_path")[1]

In [ ]:
df = pl.read_ipc(
    "/home/jqs1/scratch/jqs1/sequencing/230707_repressilators/20230707_2040_MN35044_FAS94231_25542e0d/output/default/extract_segments/*.arrow"
)

In [ ]:
df